# Source Estimation

## Import

In [1]:
import os.path as op

import mne
import os
import mne_bids
import vtk

from mne.forward import read_forward_solution
from mne.minimum_norm import (make_inverse_operator, apply_inverse,
                              write_inverse_operator)

## Directory

In [3]:
eeg_path = "eeg"
subname = "sub-"+subject
sesname = "ses-"+session
preprocessed_reports = "03_preprocessed"
forward_solution = "04_fwd"
covariance_reports = "05_covariance"
inverse_operator = "06_inv"
source_estimates = "07_stc"

In [4]:
#Makes EEG_OUTPUTS -> subject -> session -> eeg -> evoked_reports
if not os.path.exists(os.path.join(output_path, subname, sesname, 
                                   eeg_path, inverse_operator)):
    os.makedirs(os.path.join(output_path, subname, sesname, 
                             eeg_path, inverse_operator))

In [5]:
#Makes EEG_OUTPUTS -> subject -> session -> eeg -> evoked_reports
if not os.path.exists(os.path.join(output_path, subname, sesname, 
                                   eeg_path, source_estimates)):
    os.makedirs(os.path.join(output_path, subname, sesname, 
                             eeg_path, source_estimates))

In [6]:
preprocessed_report_path = os.path.join(root_path, "EGI_OUTPUTS\\"+subname+"\\"+sesname+"\\"+eeg_path+"\\"+preprocessed_reports)
forward_path = os.path.join(root_path, "EGI_OUTPUTS\\"+subname+"\\"+sesname+"\\"+eeg_path+"\\"+forward_solution)
covariance_path = os.path.join(root_path, "EGI_OUTPUTS\\"+subname+"\\"+sesname+"\\"+eeg_path+"\\"+covariance_reports)
inverse_operator_path = os.path.join(root_path, "EGI_OUTPUTS\\"+subname+"\\"+sesname+"\\"+eeg_path+"\\"+inverse_operator)
source_estimates_path = os.path.join(root_path, "EGI_OUTPUTS\\"+subname+"\\"+sesname+"\\"+eeg_path+"\\"+source_estimates)

## Input Epoch Data

In [7]:
preprocessed_name = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_preprocessed_epo.fif'
preprocessed_file = os.path.join(preprocessed_report_path, preprocessed_name)
epochs = mne.read_epochs(preprocessed_file)
epochs

Reading C:\Users\trevo\OneDrive\Desktop\Lab_Files\VWM_LAB\EGI_OUTPUTS\sub-01\ses-WM1\eeg\03_preprocessed\sub-01_ses-WM1_task-isi0_preprocessed_epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 127) active
    Found the data of interest:
        t =    -250.00 ...     800.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
471 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated


Number of events,471
Events,TsD-: 119WMD+: 118bgin: 119fix+: 115
Time range,-0.250 – 0.800 sec
Baseline,-0.250 – 0.000 sec


In [8]:
info = mne.io.read_info(preprocessed_file)

    Read a total of 1 projection items:
        Average EEG reference (1 x 127) active


## Input Forward Solution

In [9]:
fwd_name = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_fwd.fif'
fwd_file = os.path.join(forward_path, fwd_name)
fwd = mne.read_forward_solution(fwd_file)
fwd

Reading forward solution from C:\Users\trevo\OneDrive\Desktop\Lab_Files\VWM_LAB\EGI_OUTPUTS\sub-01\ses-WM1\eeg\04_fwd\sub-01_ses-WM1_task-isi0_fwd.fif...
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read EEG forward solution (20484 sources, 129 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame


Good channels,129 EEG
Bad channels,None
Source space,Surface with 20484 vertices
Source orientation,Free


## Input Noise Covariance

In [10]:
cov_name = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_cov.fif'
cov_file = os.path.join(covariance_path, cov_name)
noise_cov = mne.read_cov(cov_file)
noise_cov

    127 x 127 full covariance (kind = 1) found.
    Read a total of 1 projection items:
        Average EEG reference (1 x 127) active


<Covariance | size : 127 x 127, n_samples : 118220, data : [[ 2.05735827e-11  1.50414641e-11  8.23090641e-12 ...  1.39309335e-11
   9.93285790e-12 -1.00589750e-11]
 [ 1.50414641e-11  3.08361497e-11  1.13943832e-11 ...  9.07722232e-12
   5.83043618e-12 -6.84139971e-12]
 [ 8.23090641e-12  1.13943832e-11  2.13271530e-11 ...  4.36499316e-12
   1.96613265e-12 -3.18358192e-13]
 ...
 [ 1.39309335e-11  9.07722232e-12  4.36499316e-12 ...  2.51869090e-11
   1.94574642e-11 -1.29614305e-11]
 [ 9.93285790e-12  5.83043618e-12  1.96613265e-12 ...  1.94574642e-11
   1.97499477e-11 -1.24010238e-11]
 [-1.00589750e-11 -6.84139971e-12 -3.18358192e-13 ... -1.29614305e-11
  -1.24010238e-11  2.16403061e-11]]>

## Inverse Operator

In [11]:
inverse_operator = make_inverse_operator(info, fwd, noise_cov,
                                         loose=0.2, depth=0.8)

Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 126 channels.
    126 out of 129 channels remain after picking
Selected 126 channels
Creating the depth weighting matrix...
    126 EEG channels
    limit = 20485/20484 = 1.768882
    scale = 274009 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 1.8e-13 (2.2e-16 eps * 126 dim * 6.4  max singular value)
    Estimated rank (eeg): 102
    EEG: rank 102 computed from 126 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computin

### Write Inverse Operator

In [12]:
inv_title = 'sub-'+subject+'_ses-'+session+'_task-'+task+'-inv.fif'

inv_path = os.path.join(inverse_operator_path, inv_title)
mne.minimum_norm.write_inverse_operator(inv_path, inverse_operator, overwrite=True)

Overwriting existing file.
Write inverse operator decomposition in C:\Users\trevo\OneDrive\Desktop\Lab_Files\VWM_LAB\EGI_OUTPUTS\sub-01\ses-WM1\eeg\06_inv\sub-01_ses-WM1_task-isi0-inv.fif...
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
    Writing inverse operator info...
    Writing noise covariance matrix.
    Writing source covariance matrix.
    Writing orientation priors.
    [done]


## Calculate Source Estimate

##### Note: There are many, many different methods to use for source estimates.

In [13]:
print('######################## Computing source estimate. ########################')

######################## Computing source estimate. ########################


In [14]:
method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2

stc = apply_inverse(epochs.average(), inverse_operator, lambda2,
                    method=method, pick_ori=None)

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 471
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 102 (24 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Applying inverse operator to "0.25 × TsD- + 0.25 × WMD+ + 0.25 × bgin + 0.24 × fix+"...
    Picked 126 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained  57.6% variance
    Combining the current components...
    dSPM...
[done]


### Show Source Estimate

## Report Source Estimate

In [ ]:
stc_report_title = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_stc_report.html'
stc_report_path = os.path.join(source_estimates_path, stc_report_title)


stc_report = mne.Report(title='Source Estimate')
stc_report.add_stc(
    stc=stc, subject='fsaverage',
    title='Source Estimate', n_time_points=6  # few for speed
)
stc_report.save(stc_report_path, overwrite=True)

## Save Source Estimate

In [ ]:
stc_title = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_source_estimate.stc'

stc_path = os.path.join(source_estimates_path, stc_title)
stc.save(stc_path, overwrite=True)